In [ ]:
!pip install pythainlp tltk rapidfuzz

In [1]:
import pandas as pd
from tqdm import tqdm
from pythainlp.transliterate import transliterate
from rapidfuzz.fuzz import partial_ratio
import re

engine = 'tltk_ipa'

# === Enhanced Extraction Function ===
def extract_name_candidate(original_sentence):
    start_keywords = r'(?:ชื่อจริงชื่อ|ชื่อกระผม|กระผมชื่อ|ข้าพเจ้า|กระผม|ชื่อดิฉัน|ดิฉันชื่อ|ชื่อผม|ชื่อฉัน|เราชื่อ|ผมชื่อ|ชื่อชั้น|ดิฉัน|ดิชั้น|อิฉัน|ผม|ชื่อ|ฉัน|หนู)'
    end_keywords = r'(?:นะคะ|นะครับ|ครับ|ค่ะ|คะ|เป็น|จาก|คือ|หรือ|ชื่อเล่น|สั้นๆว่า|เรียกว่า|เรียก)'
    exclude_words = ['แนะนำ','ก็ได้','หมายเลข', 'ลงทุน', 'ใบอนุญาต', 'เลขที่','ช่วย','ปรึกษา','สำคัญ','คุย','0','1','2','3','4','5','6','7','8','9']

    pattern = re.compile(f'{start_keywords}\s*(.*?)(?=\s*{end_keywords})', re.IGNORECASE)
    match = pattern.search(original_sentence)

    if match:
        middle_text = match.group(1).strip()
        word_index = int(original_sentence.index(middle_text))
        middle_text = re.sub(r'\bชื่อ\b', '', middle_text)
        middle_text = re.sub(r'\s+', '', middle_text)
        if len(middle_text) > 10 and not any(ex_word in middle_text for ex_word in exclude_words):
            return middle_text
    else:
        if len(original_sentence) > 50:
            return original_sentence[:50]
        return original_sentence
# === Name Matching Function ===
def name_rule_based(first_name, last_name, transcription, threshold=65, debug=False):
    # Clean and transliterate
    try:
        first_name_phoneme = transliterate(re.sub(r'\s+', '', first_name), engine=engine).replace(" ", "").lower()
        last_name_phoneme = transliterate(re.sub(r'\s+', '', last_name), engine=engine).replace(" ", "").lower()
        transcription_phoneme = transliterate(re.sub(r'\s+', '', transcription), engine=engine).replace(" ", "").lower()

        # Fuzzy match
        first_score = partial_ratio(transcription_phoneme, first_name_phoneme)
        last_score = partial_ratio(transcription_phoneme, last_name_phoneme)
        transcription = transcription.replace('ชื่อ','')
        if debug:
            print("DEBUG:")
            print(f"{first_name} ({first_score}) => {first_name_phoneme}")
            print(f"{last_name} ({last_score}) => {last_name_phoneme}")
            print(f"Transcript => {transcription_phoneme}")
            print(f"{transcription[:90]}")
        if 13 < len(transcription) < 50 :
            # print(True)
            return True
        elif first_score > 60 and last_score >= 62:
            # print(True)
            return True
        else:
            # print(False)
            return False
    except:
        print("ERROR")
        print(transcription)

# === Load Data ===
post_df = pd.read_csv("/data/Calvin_007/week4/Chen_Pre_for_Post.csv")  # <-- มี id,transcription,firstname,lastname (audio,sentence,first_name,last_name)
# === Prepare Output Columns ===
post_df["ชื่อที่ตรวจพบ"] = ""
post_df["แนะนำชื่อและนามสกุล"] = False
# === Main Processing Loop ===
for row in (post_df.itertuples(index=True)):
    transcription = row.transcript
    first_name = row.first_name
    last_name = row.last_name

    # Try to extract candidate name
    candidate_name = extract_name_candidate(transcription)
    post_df.at[row.Index, "ชื่อที่ตรวจพบ"] = candidate_name if candidate_name else ""
    if candidate_name:
        # Rule-based name match
        matched = name_rule_based(first_name, last_name, candidate_name, threshold=65,debug = False)
        post_df.at[row.Index, "แนะนำชื่อและนามสกุล"] = matched

mandojin = pd.read_csv('/data/Calvin_007/week4/submission_week4_dojin224.csv')  # <-- มี ไฟล์คำตอบจาก Model
merge_df = post_df.merge(mandojin[['id', 'แนะนำชื่อและนามสกุล', 'บอกระยะเวลาที่ใช้ในการเข้าพบ']], on='id', suffixes=('_post', '_man'))
ans = merge_df[['id','แนะนำชื่อและนามสกุล_post']]
mandojin['แนะนำชื่อและนามสกุล'] = mandojin['id'].map(ans.set_index('id')['แนะนำชื่อและนามสกุล_post'])
mandojin = mandojin.sort_values('id').reset_index(drop=True)
mandojin.to_csv('./api_dojin.csv')